# PopGPT on Paperspace
Just click "START MACHINE" to run your new PopTorch code on the IPU!

In [ ]:
import torch
import torchvision
import torchvision.transforms as transforms
import poptorch

transform = transforms.Compose(
    [transforms.ToTensor(), transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))]
)

batch_size = 4

trainset = torchvision.datasets.CIFAR10(
    root=\"./data\", train=True, download=True, transform=transform
)

testset = torchvision.datasets.CIFAR10(
    root=\"./data\", train=False, download=True, transform=transform
)

classes = (
    \"plane\",
    \"car\",
    \"bird\",
    \"cat\",
    \"deer\",
    \"dog\",
    \"frog\",
    \"horse\",
    \"ship\",
    \"truck\",
)

opts = poptorch.Options()
trainloader = poptorch.DataLoader(
    options=opts,
    dataset=trainset,
    batch_size=batch_size,
    shuffle=True,
    num_workers=2,
)

val_opts = poptorch.Options()
testloader = poptorch.DataLoader(
    options=val_opts,
    dataset=testset,
    batch_size=batch_size,
    shuffle=False,
    num_workers=2,
)

import torch.nn as nn
import torch.nn.functional as F


class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = torch.flatten(x, 1)  # flatten all dimensions except batch
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x


net = Net()

import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

# Wrap the model for training
poptorch_model = poptorch.trainingModel(net, options=opts, optimizer=optimizer)

for epoch in range(2):  # loop over the dataset multiple times
    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data

        # forward + backward + optimize
        outputs = poptorch_model(inputs, labels)
        loss = criterion(outputs, labels)

        # print statistics
        running_loss += loss.item()
        if i % 2000 == 1999:  # print every 2000 mini-batches
            print(f\"[{epoch + 1}, {i + 1:5d}] loss: {running_loss / 2000:.3f}\")
            running_loss = 0.0

print(\"Finished Training\")

PATH = \"./cifar_net.pth\"
torch.save(net.state_dict(), PATH)

# Wrap the model for inference
poptorch_inference_model = poptorch.inferenceModel(net, options=val_opts)

correct = 0
total = 0
# since we're not training, we don't need to calculate the gradients for our outputs
with torch.no_grad():
    for data in testloader:
        images, labels = data
        # calculate outputs by running images through the network
        outputs = poptorch_inference_model(images)
        # the class with the highest energy is what we choose as prediction
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f\"Accuracy of the network on the 10000 test images: {100 * correct // total} %\")
